# Python for Data Science 101
by Adam Raveret

# 1. Environment Setup

<img src="images/anaconda_logo.png" style="float: none; margin: 10px 10px 0 0; width: 200px;"/>

**Navigate to:** https://www.anaconda.com/download/

**What is it?** Anaconda is a free and open-source distribution of the Python that aims to simplify package management and deployment. It includes the most commonly used libraries, as well as pip, the Python package manager used to install and manage new libraries.

**Step 1**

<img src="images/image_01a.png" style="width: 800px;"/>

**Step 2**
<img src="images/image_01b.png" style="width: 800px;"/>

**Step 3**
<img src="images/image_01c.png" style="width: 800px;"/>

<img src="images/git_logo.png" style="float: none; margin: 15px 15px 0 0; width: 100px;"/>

**Navigate to:** https://git-scm.com/download

**What is it?** Git is a distributed version control system for tracking changes in source code during software development. It is designed for coordinating work among programmers, but it can be used to track changes in any set of files.

<img src="images/image_02.png" style="width: 800px;"/>

<img src="images/github_logo.png" style="float: none; margin: 15px 15px 0 0; width: 200px;"/>

**Navigate to:** https://github.com/

**What is it?**  Github is a web-based hosting service for version control using Git. It offers all of the distributed version control and source code management functionality of Git as well as access control and several collaboration features such as bug tracking, feature requests, task management, and wikis for every project

<img src="images/image_03.png" style="width: 800px;"/>

<img src="images/aws_logo.png" style="float: none; margin: 15px 15px 0 0; width: 200px;"/>

**Navigate to:** https://aws.amazon.com/free/

**What is it?** Amazon Web Services (AWS) is a subsidiary of Amazon that provides on-demand cloud computing platforms to individuals, companies and governments, on a paid subscription basis. The technology allows subscribers to have at their disposal a virtual cluster of computers, available all the time, through the Internet.

<img src="images/image_04.png" style="width: 800px;"/>

<img src="images/gcp_logo.png" style="float: none; margin: 15px 15px 0 0; width: 250px;"/>

**Navigate to:** https://cloud.google.com/

**What is it?** Google Cloud Platform (GCP) is a suite of cloud computing services that runs on the same infrastructure that Google uses internally for its end-user products, such as Google Search and YouTube. Alongside a set of management tools, it provides a series of modular cloud services including computing, data storage, data analytics and machine learning.

<img src="images/image_05.png" style="width: 800px;"/>

## Required Packages
Using the command line, navigate to the directory where you want to store this training:

`cd`

After you have copied the HTTPS clone link, run the command:

`git clone https://github.com/araveret/training.git`

Navigate into your new repository and install the required packages with the command: 

`pip install -r requirements.txt`

# 2. Base Python
Using the command line, open Spyder with the command: 

`spyder`

Once Spyder opens, drag the 'base_python.py' file into the text editor on the left side of the integrated developer environment (IDE). 

# 3. Data Collection
## Flat file
**Use Case:** Titanic (CSV)

**Exercise**

In [ ]:
# load the titanic data in the data folder
path = r'./data/titanic.csv'
titanic = 

In [ ]:
titanic.columns = ['survived', 'pclass', 'name', 'sex', 'age', 'sib_spouse', 'parent_child', 'fare']

## Database
**Use Case:** Titanic (AWS RDS) 

In [ ]:
# import libraries
import psycopg2
from io import StringIO

# create a function to create a dictionary which contains all the information you need in a create statement
def create_dict(df):
    dct = {}
    for n,i in enumerate(df.dtypes):
        dct[df.columns[n]] = [str(i),str(max([len(x) for x in df[df.columns[n]].astype(str)]))]
    return dct

In [ ]:
# create the dictionary, the name of the table, and a key for columns types
dct = create_dict(titanic)
table = 'titanic'
key = {'object':'varchar',
       'int64':'int',
       'float64':'float',
       'datetime64[ns]':'date',
       'bool':'boolean'
       }

# build your create statement
create = "DROP TABLE IF EXISTS "+table+" CASCADE; CREATE TABLE "+table+"("+ \
    ', '.join([item+' '+key[dct[item][0]]+'('+dct[item][1]+')' 
               if ((key[dct[item][0]]!='date')&(key[dct[item][0]]!='int')) 
               else item+' '+key[dct[item][0]] for item in titanic.columns]) \
    +');'

# initialize a string buffer
sio = StringIO()
sio.write(titanic.to_csv(index=None, header=None))
sio.seek(0)
#sio.getvalue()

In [ ]:
# create a connection with the database
conn = psycopg2.connect(' '.join(['host=PUT_HOST_HERE',
                                  'dbname=PUT_DBNAME_HERE',
                                  'user=PUT_USER_HERE',
                                  'password=PUT_PASS_HERE']))

# create a table to store your data
with conn.cursor() as c:
    c.execute(create)
    conn.commit()

# copy the string buffer to the table
with conn.cursor() as c:
    c.copy_from(sio, "titanic", columns=titanic.columns, sep=',')
    conn.commit()

# close the connection with the database
conn.close()

In [ ]:
# create a connection with the database
conn = psycopg2.connect(' '.join(['host=PUT_HOST_HERE',
                                  'dbname=PUT_DBNAME_HERE',
                                  'user=PUT_USER_HERE',
                                  'password=PUT_PASS_HERE']))

# fetch and store all data from the table
with conn.cursor() as c:
    c.execute('SELECT * FROM '+table)
    data = c.fetchall()
    
# close the conenction with the database
conn.close()

# convert stored data into a Pandas dataframe
df2 = pd.DataFrame(data, columns=titanic.columns)

## Webscraping 
**Use Case:** Python.org Events (HTML)

In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup

# request the html code from a url
url = r'https://www.python.org/events/python-events/'
r = requests.get(url)

In [ ]:
# convert HTML into a structured Soup object
b = BeautifulSoup(r.text, 'lxml')

In [ ]:
# get the text for one element in the table
b.find_all('ul', attrs={'class':"list-recent-events menu"})[0]

In [ ]:
# store all of the elements by column in a dictionary
dct = {'event':[x.text for x in b.find_all('ul', attrs={'class':"list-recent-events menu"})[0].find_all('h3')],
       'date':[x.text for x in b.find_all('ul', attrs={'class':"list-recent-events menu"})[0].find_all('time')],
       'location':[x.text for x in b.find_all('ul', attrs={'class':"list-recent-events menu"})[0].find_all('span', attrs={'class':'event-location'})]
      }

# convert the dictionary into a data table
events = pd.DataFrame(dct)

**Exercise**

In [ ]:
# repeat the above process to create a dataframe containing information about recent polls
url = r'https://www.realclearpolitics.com/epolls/latest_polls/'


## API
**Use Case:** Google Maps (REST)

In [ ]:
import urllib.parse
import requests
import json

# part 1 - base path of the api
main = r'https://maps.googleapis.com/maps/api/geocode/json?'

# part 2 - the address search component
search = '555 Hamilton Avenue, Palo Alto, CA'

# part 3 - your api developer key to authenticate the search
api_key = r'&key=API_KEY_HERE'

# put parts 1, 2, and 3 together
url = main + urllib.parse.urlencode({'address':search})+api_key

# make a get request to the api
map_data = requests.get(url).json()

In [ ]:
# using the Python client 
import googlemaps

gmaps = googlemaps.Client(key=r'API_KEY_HERE')
search = ('555 Hamilton Avenue, Palo Alto, CA')
map_data = gmaps.geocode(search)

**Exercise**

In [ ]:
# explore the Star Wars API to find the name of the "homeworld" of "Han Solo"
url = r'http://swapi.co/api/'


# 4. ETL
## Data Manipulation

In [ ]:
# add a column for each passenger's title
titanic['title'] = [x.split('.')[0] for x in titanic['name']]

In [ ]:
# add a column to group each passenger by age range
age_group = []

for age in titanic['age']:
    if age < 1:
        age_group.append('00 years')
    elif age <= 4:
        age_group.append('01-04 years')
    elif age <= 9:
        age_group.append('05-09 years')
    elif age <= 14:
        age_group.append('10-14 years')
    elif age <= 19:
        age_group.append('15-19 years')
    elif age <= 24:
        age_group.append('20-24 years')
    elif age <= 29:
        age_group.append('25-29 years')
    elif age <= 34:
        age_group.append('30-34 years')
    elif age <= 39:
        age_group.append('35-39 years')
    elif age <= 44:
        age_group.append('40-44 years')
    elif age <= 49:
        age_group.append('45-49 years')
    elif age <= 54:
        age_group.append('50-54 years')
    else:
        age_group.append('55+ years')
        
titanic['age_group'] = age_group

## Airflow

To install Aiflow on your local machine, enter in your terminal:

`pip install apache-airflow`

Once Airflow has installed, you will need to initiate a database which stores all of the information Airflow needs to run:

`airflow initdb`

Store the 'titanic_example.py' file in your Airflow directory:

`Users/[YOUR_USERNAME]/airflow/dags/titanic_example.py`

Create a 'stage' directory beside the 'dags' folder to store stage data:

`Users/[YOUR_USERNAME]/airflow/stage`

Next, run the Airflow scheduler in your terminal:

`airflow scheduler`

Then, in a new terminal window, run the Airflow webserver:

`airflow webserver -p 8080`

When you are finished with Airflow you can close the scheduler and kill the webserver with the following command:

`cat airflow/airflow-webserver.pid | xargs kill -9`

# 5. Statistical Modeling
## Introduction

## Regression

**What is it?**

Linear regression is a linear approach to modelling the relationship between a scalar response (or dependent variable) and one or more explanatory variables (or independent variables).

**How do you do it?**

<img src="images/regression_1.png" style="width: 800px;"/>

**How do you evaluate it?**

<img src="images/regression_2.png" style="width: 800px;"/>

## Classification

**What is it?**

Classification is the problem of identifying to which of a set of categories (sub-populations) a new observation belongs, on the basis of a training set of data containing observations (or instances) whose category membership is known.

**How do you do it?**

**How do you evaluate it?**

## Clustering

**What is it?**

Clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense) to each other than to those in other groups (clusters).

**How do you do it?**

**How do you evaluate it?**

## Dimensionality Reduction

**What is it?**

Dimensionality reduction is the process of reducing the number of random variables under consideration by obtaining a set of principal variables. It can be divided into feature selection and feature extraction.

**How do you do it?**

**How do you evaluate it?**

## Example (Classification)

In [ ]:
from sklearn import tree
from sklearn import ensemble
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
y = titanic['survived']

In [ ]:
pclass = pd.get_dummies(titanic['pclass'], prefix='pclass').iloc[:,:-1]
sex = pd.get_dummies(titanic['sex'], prefix='sex').iloc[:,:-1]
title = pd.get_dummies(titanic['title'], prefix='title').iloc[:,:-1]
age_group = pd.get_dummies(titanic['age_group'], prefix='age_group').iloc[:,:-1]
X = pd.concat([pclass, sex, title, age_group], axis=1)

In [ ]:
dtree = tree.DecisionTreeClassifier(random_state=1)
depth_range = list(range(1,20))
param_grid = dict(max_depth=depth_range)
grid = GridSearchCV(dtree, param_grid, cv=5, scoring='roc_auc')
grid.fit(X, y)

In [ ]:
grid_mean_scores = grid.cv_results_['mean_test_score']

In [ ]:
plt.figure()
plt.plot(depth_range, grid_mean_scores)
plt.grid(True)
plt.plot(grid.best_params_['max_depth'], grid.best_score_, 'ro', markersize=12, markeredgewidth=1.5, markerfacecolor='None', markeredgecolor='r')

In [ ]:
best = grid.best_estimator_

In [ ]:
pd.DataFrame(best.feature_importances_, index=X.columns.tolist())[0].sort_values(ascending=False)

In [ ]:
preds = best.predict(X)

In [ ]:
metrics.accuracy_score(y, preds)

In [ ]:
metrics.confusion_matrix(y, preds)

In [ ]:
probs = best.predict_proba(X)[:,1]

In [ ]:
metrics.roc_auc_score(y, probs)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y, probs)
plt.plot(fpr, tpr)
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('FPR (1 - Specificity)')
plt.ylabel('TPR (Sensitivity)')

**Exercise**

In [ ]:
# compare the above model to a Random Forest Classifier tuning the 'n_estimators' and 'max_depth' parameters
rforest = ensemble.RandomForestClassifier(random_state=1)


# 6. Model Deployment
## Pickle

In [ ]:
import pickle
filename = r'./model/dtree_model.pkl'
pickle.dump(best, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
metrics.accuracy_score(y, loaded_model.predict(X))

## AWS Endpoint

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/random_cut_forest/random_cut_forest.ipynb

# 7. Web Application
## Flask